# VM translator based on elements of computing systems

In [1]:
import sys, os
from io import StringIO
from typing import List, Dict, Tuple
from enum import Enum
from functools import partial

# sys.path.append(os.path.join(os.path.dirname(__file__), "..", "lib"))
from lib.utils import pretty_format_dict, binary_or, binary_and, binary_add, binary_flip, binary_neg, WORD_SIZE

CommandType = Enum(
    'CommandType', 
    ['C_ARITHMETIC', # page 130, fig 7.5: e.g., add, sub, neg ...
     'C_PUSH', # page 131: push <segment> index, e.g., push argument 0 // stack.push(argument[0])
     'C_POP', # page 131: pop <segment> index, e.g., pop argment 0 // argment[0] = stack.pop()
     'C_LABEL', # page 159: label symbol, marks location in code, scope is within the function
     'C_GOTO',  # page 159: goto label, unconditional jump
     'C_IF', # page 159: if-goto label, pc = label if stack.pop() != 0 else pc + 1, label must be within the same function
     'C_FUNCTION', # page 163: function f k, where k is num local variables
     'C_RETURN', # page 163: return, return control to the caller
     'C_CALL', # page 163: call f n, where f is a function and n is number of arguments
    ]
)

ARITHMETIC_COMMANDS = ['add', 'sub', 'neg', 'eq', 'gt', 'lt', 'and', 'or', 'not']

def getCommandType(command: str)->CommandType:
    if command in ARITHMETIC_COMMANDS:
        return CommandType.C_ARITHMETIC
    elif command.startswith('push'):
        return CommandType.C_PUSH
    elif command.startswith('pop'):
        return CommandType.C_POP
    elif command.startswith('label'):
        return CommandType.C_LABEL
    elif command.startswith('goto'):
        return CommandType.C_GOTO
    elif command.startswith('if-goto'):
        return CommandType.C_IF
    elif command.startswith('function'):
        return CommandType.C_FUNCTION
    elif command.startswith('return'):
        return CommandType.C_RETURN
    elif command.startswith('call'):
        return CommandType.C_CALL
    
class Interpretor: 
    # AKA python machine that runs VM code
    '''VM interpretor, essentially a simulator in python
    instead of compiling the code, just run on the fly instead
    this is like the class Machine in assembler

    Here we assume VM only contain functions

    Interpreter is essentially a python machine
    where the hardware (including ROM) is specified by self.machine
    and machine language is python (the generated code is the concatenation
    of all the text in code execution parts of self.advance)
    '''
    def __init__(self, 
                 multi_line_comment_open:str='/*',
                 multi_line_comment_close:str='*/',
                 heap_size:int=5, static_size:int=5, temp_size:int=5, 
                 max_steps:int=100, verbose:bool=True):
        self.multi_line_comment_open = multi_line_comment_open
        self.multi_line_comment_close = multi_line_comment_close
        self.temp_size = temp_size
        self.static_size = static_size
        self.heap_size = heap_size
        self.max_steps = max_steps
        self.verbose = verbose
        
    def load(self, vm_fnames: List[str]):
        self.machine = {
            'functions': {},
            'current_function': "Sys.init", # always start from Sys.init
            'pc': 0, # line number within current_function
            'stack': [], # page 141 M[256:2048]
            'heap': [0]*self.heap_size, # for storing objects, page 141 M[2048:16383]
            # 'sp': 0 # stack pointer: next top location in stack, page 142 R0, this need to be separately implemneted for hack machine but not for python
            'segments': {
                'argument': 0, # dynamically allocated per function, point to a cell on stack, page 142 R2
                'local': 0, # dynamically allocated per function, point to a cell on stack, page 142 R1
                'static': [0] * self.static_size, # shared by all functions in .vm file, page 141 M[16:256]
                # 'constant': [], # psshared by all functions
                # 'this': [], # pointer to heap: pointer[0]
                # 'that': [], # pointer to heap: pointer[1]
                'pointer': [0, 0], # pointer to this and that, page 142 M[3:5]
                'temp': [0] * self.temp_size, # shared by all functions, page 142 M[5:13]
            }
        }

        # set up symbol lookup: symbol['current_function']['label'] -> 'pc'
        # b/c label in VM is attached to the function scope
        self.symbol_table = {}
        
        # populate functions and symbol table
        for fname in vm_fnames:
            with open(fname) as f:
                self._first_pass(f.readlines())

        if self.verbose:
            print('functions parsed')
            print(pretty_format_dict(self.machine['functions']))
            print('symbol table')
            print(pretty_format_dict(self.symbol_table))

        # make sure Sys.init is encountered
        assert self.machine['current_function'] in self.machine['functions'], f'{self.machine["current_function"]} need to be in vm code'

    def __repr__(self):
        return f'VM interpretor(Machine=\n{pretty_format_dict(self.machine)}\nSymbolTable=\n{pretty_format_dict(self.symbol_table)}\n)'

    def _first_pass(self, codes: List[str], in_comment:bool=False):
        # parse out all the functions into self.machine['functions'], add labels and function to symbol table,
        # and strip out comments
        # in_comment: are you in multi line comment /* */?
        func_name, func_codes = None, []
        lineno_within_func = 0

        for i, l in enumerate(codes):

            if in_comment:
                if self.multi_line_comment_close in l:
                    # comment closed
                    len_c_close = len(self.multi_line_comment_close)
                    l = l[l.index(self.multi_line_comment_close)+len_c_close:]
                    in_comment = False
                else:
                    # comment continue to open, ignore the line
                    continue
                    
            # sanitize line
            l = l.strip()
            if '//' in l: # comment
                l = l[:l.index('//')].strip()

            # not in multiline comment
            if self.multi_line_comment_open in l:
                in_comment = True
                l = l[:l.index(self.multi_line_comment_open)].strip()
                
            # scrape function
            ct = getCommandType(l)
            if ct is CommandType.C_FUNCTION:
                if func_name is not None:
                    self.machine['functions'][func_name] = func_codes
                    return self._first_pass(codes[i:])
                else:
                    func_name = l.split()[1]
                    lineno_within_func += 1
                    if l != "": func_codes.append(l)

            elif ct is CommandType.C_LABEL:
                # 'C_LABEL', # page 159: label symbol, marks location in code, scope is within the function
                assert func_name is not None, f'label {l} need to be within a function'
                label = l.split()[1]
                if func_name not in self.symbol_table:
                    self.symbol_table[func_name] = {}
                self.symbol_table[func_name][label] = lineno_within_func
            else:
                lineno_within_func += 1
                if l != "": func_codes.append(l)
                
        if func_name is not None:
            self.machine['functions'][func_name] = func_codes
            return


    def advance(self)->bool:
        func_name, pc = self.machine['current_function'], self.machine['pc']
        codes = self.machine['functions'][func_name]

        if self.verbose:
            print('stack:', self.machine['stack'])
        
        if pc >= len(codes):
            # finished execution
            return False

        code = codes[pc]
        ct = getCommandType(code)
        if self.verbose:
            print(f'instruction {func_name}[{pc}]: {code}, type: {ct}')
        
        self.machine['pc'] += 1
        # instruction decoding and then execute
        stack = self.machine['stack']
        heap = self.machine['heap']
        segments = self.machine['segments']
        
        if ct is CommandType.C_ARITHMETIC:
            # 'C_ARITHMETIC', # page 130, fig 7.5: e.g., add, sub, neg ...
            x = self.machine['stack'].pop()
            if code in ['add', 'sub', 'eq', 'gt', 'lt', 'and', 'or']:
                # binary operators, TODO: rewrite some of them to bitwise operations 
                y = self.machine['stack'].pop()
                ret = {
                    'add': partial(binary_add, word_size=WORD_SIZE),
                    'sub': lambda a, b: a - b,
                    'eq': lambda a, b: a == b,
                    'gt': lambda a, b: a > b,
                    'lt': lambda a, b: a < b,
                    'and': partial(binary_and, word_size=WORD_SIZE),
                    'or': partial(binary_or, word_size=WORD_SIZE),
                }[code](x, y)
            else: 
                # unary operators
                ret = {
                    'neg': partial(binary_neg, word_size=WORD_SIZE),
                    'not': partial(binary_flip, word_size=WORD_SIZE),
                }[code](x)
                    
            self.machine['stack'].append(ret)
        elif ct is CommandType.C_PUSH:
            # 'C_PUSH', # page 131: push <segment> index, e.g., push argument 0 // stack.push(argument[0])
            _, segment, index = code.split()
            index = int(index)
            
            if segment == 'argument':
                # dynamically allocated per function, point to a cell on stack
                val = stack[segments[segment] + index]
            elif segment == 'local':
                # dynamically allocated per function, point to a cell on stack
                val = stack[segments[segment] + index]
            elif segment == 'static':
                # shared by all functions in .vm file, page 141 M[16:256]
                val = segments[segment][index]
            elif segment == 'constant':
                # shared by all functions
                val = index
            elif segment == 'this':
                # pointer to heap: pointer[0]
                val = heap[segment['pointer'][0] + index]
            elif segment == 'that':
                # pointer to heap: pointer[1]
                val = heap[segment['pointer'][1] + index]
            elif segment == 'pointer':
                # pointer to this and that, page 142 M[3:5]
                val = self.machine[segment][index]
            elif segment == 'temp':
                # shared by all functions, page 142 M[5:13]
                val = self.machine[segment][index]
            else:
                assert False, f'unknown segment {segment}'
                
            self.machine['stack'].append(val)
            
        elif ct is CommandType.C_POP:
            # 'C_POP', # page 131: pop <segment> index, e.g., pop argment 0 // argment[0] = stack.pop()
            _, segment, index = code.split()
            index = int(index)

            val = self.machine['stack'].pop()
            if segment == 'argument':
                # dynamically allocated per function, point to a cell on stack
                stack[segments[segment] + index] = val
            elif segment == 'local':
                # dynamically allocated per function, point to a cell on stack
                stack[segments[segment] + index] = val
            elif segment == 'static':
                # shared by all functions in .vm file, page 141 M[16:256]
                segments[segment][index] = val
            elif segment == 'constant':
                # shared by all functions
                val = index
            elif segment == 'this':
                # pointer to heap: pointer[0]
                val = heap[segment['pointer'][0] + index]
            elif segment == 'that':
                # pointer to heap: pointer[1]
                val = heap[segment['pointer'][1] + index]
            elif segment == 'pointer':
                # pointer to this and that, page 142 M[3:5]
                val = self.machine[segment][index]
            elif segment == 'temp':
                # shared by all functions, page 142 M[5:13]
                val = self.machine[segment][index]
            else:
                assert False, f'unknown segment {segment}'

        elif ct is CommandType.C_LABEL:
            # 'C_LABEL', # page 159: label symbol, marks location in code, scope is within the function
            assert False, f'{ct} command should only be encountered in _first_pass, as in this second pass it should have been resolved'
        elif ct is CommandType.C_GOTO:
            # 'C_GOTO',  # page 159: goto label, unconditional jump
            label = code.split()[1]
            assert label in self.symbol_table[self.machine['current_function']], f'{label} not in current function scope'
            self.machine['pc'] = self.symbol_table[self.machine['current_function']][label]
        elif ct is CommandType.C_IF: 
            # 'C_IF', # page 159: if-goto label, pc = label if stack.pop() != 0 else pc + 1, label must be within the same function
            label = code.split()[1]
            label_pc = self.symbol_table[self.machine['current_function']][label]
            if self.machine['stack'].pop() != 0:
                self.machine['pc'] = label_pc # no need to consider = 0 case b/c that's already default
        elif ct is CommandType.C_FUNCTION:
            # 'C_FUNCTION', # page 163: function f k, where k is num local variables
            n_local = int(code.split()[2])
            for _ in range(n_local):
                stack.append(0)
        elif ct is CommandType.C_RETURN:
            # 'C_RETURN', # page 163: return, return control to the caller
            frame = segments['local']
            prev_f, prev_f_lineno = stack[frame-5] # ret_addr for python machine it should be (), but for hack it will be a label to ROM
            # put return value back at the new top: see page 162, here I think the assumption is a function always returns something, could be None
            stack[segments['argument']] = stack.pop()
            sp = segments['argument'] + 1
            segments['pointer'] = [stack[frame-2], stack[frame-1]]
            segments['argument'] = stack[frame-3]
            segments['local'] = stack[frame-4]
            self.machine['stack'] = self.machine['stack'][:sp]

            # goto prev function
            self.machine['current_function'] = prev_f
            self.machine['pc'] = prev_f_lineno
        elif ct is CommandType.C_CALL:
            # 'C_CALL', # page 163: call f n, where f is a function and n is number of arguments
            _, next_f, n_args = code.split()
            n_args = int(n_args)
            # push return-address
            curr_f, pc = self.machine['current_function'], self.machine['pc']
            stack.append((curr_f, pc))
            # push lcl
            stack.append(segments['local'])
            # pushl arg
            stack.append(segments['argument'])
            # push this and that
            stack.extend(segments['pointer'])
            # arg = SP - n - 5
            segments['argument'] = len(stack) - n_args - 5
            # lcl = SP
            segments['local']= len(stack)
            # goto f
            self.machine['current_function'] = next_f
            self.machine['pc'] = 0
        else:
            assert False, f'command {ct} not founc'
        
        return True

    
    def __call__(self, vm_fnames: List[str]):
        self.load(vm_fnames)

        steps = 0
        while steps <= self.max_steps:
            if self.verbose:
                print('Machine step', steps)
            steps += 1
            ok = self.advance()
            if not ok:
                print('Finished execution')
                return
        print(f'Program terminated b/c exceeding max step of {self.max_steps}')        

vm_interpretor = Interpretor()
vm_interpretor(['vm_codes/test.vm'])
# print(vm_interpretor)

functions parsed
{
    "double": [
        "function double 0",
        "push argument 0",
        "push argument 0",
        "add",
        "return"
    ],
    "Sys.init": [
        "function Sys.init 1",
        "push constant 3",
        "call double 1",
        "pop local 0"
    ]
}
symbol table
{}
Machine step 0
stack: []
instruction Sys.init[0]: function Sys.init 1, type: CommandType.C_FUNCTION
Machine step 1
stack: [0]
instruction Sys.init[1]: push constant 3, type: CommandType.C_PUSH
Machine step 2
stack: [0, 3]
instruction Sys.init[2]: call double 1, type: CommandType.C_CALL
Machine step 3
stack: [0, 3, ('Sys.init', 3), 0, 0, 0, 0]
instruction double[0]: function double 0, type: CommandType.C_FUNCTION
Machine step 4
stack: [0, 3, ('Sys.init', 3), 0, 0, 0, 0]
instruction double[1]: push argument 0, type: CommandType.C_PUSH
Machine step 5
stack: [0, 3, ('Sys.init', 3), 0, 0, 0, 0, 3]
instruction double[2]: push argument 0, type: CommandType.C_PUSH
Machine step 6
stack: [0, 3, ('

In [2]:
class Translator:
    '''VM code -> Assembly code'''
    def __init__(self):
        pass

    def __call__(self, vm_fnames: List[str])->str:
        pass

class Parser:

    def __init__(self, fname:str):
        self.fstream = open(fname, 'r')
        self.fname = fname
        self.lineno = -1 # vm line number

    def advance(self)->Tuple[bool, str, int]:
        '''
        read the next command
        return (ok, next command, reference_line_in_orig_file)
        '''
        l = self.filestream.readline()
        if l == '':
            # EOF error
            return False, '', self.lineno

        self.lineno += 1
        l = l.strip()
        # todo: maybe handle multi line comment like /* */
        if '//' in l: # comment
            l = l[:l.index('//')].strip()
            
        if l == '':    
            return self.advance()
        else:
            return True, l, self.lineno
        
        